In [9]:
options(repr.matrix.max.rows=100, repr.matrix.max.cols=300)
options(repr.plot.width = 20, repr.plot.height = 15)
options(width=300)

numcores=8

library(tidyverse)
library(parallel)
library(data.table)
library(fst)
library(comorbidity)
library(zeallot)
library(reshape)
library(dtplyr)
library(haven)
library(vroom)
library(dplyr)
`%!in%` = Negate(`%in%`)

setDTthreads(numcores)

# Codes

First, we will add codes from ICD and Medicare:primary_care_specialty_codes

In [2]:
#diagnosis codes
#from https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleId=52850&ver=26 and https://www.aapc.com/codes/cpt-codes-range/93451-93533/10
angio_codes=c(93451,93452,93453,93454,93455,93456,93457,93458,93459,93460,93461,93462,93463,93464
              ,93503,93505,93530,93531,93532,93533)
#from https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleid=57326&ver=13&keyword=electrocardiogram&keywordType=starts&areaId=all&docType=NCA,CAL,NCD,MEDCAC,TA,MCD,6,3,5,1,F,P&contractOption=all&sortBy=relevance&bc=1
ecg_codes=c(93000,93005,93010,93040,93041,93042)
#from https://scct.org/page/CardiacCTCodes include CTangio
cardiac_ct_codes=c(75571,75572,75573,75574)
#from https://cardiacmri.com/tech-guide/cpt-codes-relevant-to-cardiac-mri/
cardiac_mri_codes=c(75557,75559,75561,75563,75565)
# from https://medicarepaymentandreimbursement.com/2011/07/cardiovascular-stress-testing-cpt-93015.html and https://www.aapc.com/codes/cpt-codes-range/93000-93050/
stress_test_codes=c(93015,93016,93017,93018)
#from https://www.aapc.com/codes/cpt-codes-range/93303-93356/20     includes stress echo
echocardiography_codes=c(93303,93304,93306,93307,93308,93312,93313,93314,93315,93316,93317,93318
                         ,93320,93321,93325,93350,93351,93356,93352,93355,93356)
#from https://www.aapc.com/codes/cpt-codes-range/92920-92979/ and https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleId=57479#:~:text=CPT%20codes%2092928%2C%2092933%2C%2092929,are%20assigned%20to%20APC%200104.    includes balloon and stent
angioplasty_codes=c(92920,92921,92924,92925,92928,92929,92933,92934,92937,92938,92941,92943,92944
                    ,92973,92974,92975,92978,92979,93571,93572,"C9600","C9601","C9602","C9603"
                    ,"C9604","C9605","C9606","C9607","C9608")
#from https://www.medaxiom.com/clientuploads/webcast_handouts/Coding_for_CABG-Open_Heart_Procedures.pdf and https://www.aapc.com/codes/cpt-codes-range/33016-33999/10    did not include 33517-33530 since these are used in conjunction with 33533-33548 and not alone, did not include 33542,33545,33548 since these are also in conjunction )aneurismectomy and vsd resection
CABG_codes=c(33510,33511,33512,33513,33514,33516,33533,33534,33535,33536)
#from http://www.icd9data.com/2015/Volume1/390-459/430-438/default.htm and https://www.icd10data.com/ICD10CM/Codes/I00-I99/I60-I69/I63-
stroke_icd_9_codes=c(43301,43311,43321,43331,43381,43391,43401,43411,43491)
office_visit_codes=c("99201","99202","99203","99204","99205","99211","99212","99213","99214"
                     ,"99215")
IHD_icd_9_codes=c(410, 411, 412,413,414)
IHD_icd_10_codes=c("I20", "I21", "I22", "I23", "I24", "I25")

non_us_state_codes=c(40,54,56,57,58,59,60,61,62,63,64,65,66,97,98,99)

primary_care_specialty_codes=c("01", "08", "11", "38")
surgery_specialty_codes=c("02","04","14","19","20","24","28","33","34","40","48","77","78","85","91")

#http://www.icd9data.com/2015/Volume1/390-459/401-405/default.htm
#https://www.icd10data.com/ICD10CM/Codes/I00-I99/I10-I16
hypertension_icd_9_codes=c("401","402","403","404","405")
hypertension_icd_10_codes=c("I10","I11","I12","I13","I15","I16")

#http://www.icd9data.com/2014/Volume1/460-519/490-496/default.htm
#https://www.icd10data.com/ICD10CM/Codes/J00-J99/J40-J47
copd_icd_9_codes=c("490","491","492","493","494","495","496")
copd_icd_10_codes=c("J40","J41","J42","J43","J44","J45","J47")

#http://www.icd9data.com/2015/Volume1/240-279/270-279/278/278.htm?__hstc=93424706.cdd51240e438a5219319ce13ccb23860.1648603374124.1648603374124.1648607295327.2&__hssc=93424706.9.1648607295327&__hsfp=908776442
#https://www.icd10data.com/ICD10CM/Codes/E00-E89/E65-E68/E66-
obesity_icd_9_codes=c("278")
obesity_icd_10_codes=c("E66")

#http://www.icd9data.com/2014/Volume1/290-319/295-299/296/default.htm
#https://www.icd10data.com/ICD10CM/Codes/F01-F99/F30-F39
depression_icd_9_codes=c("2962","2963")
depression_icd_10_codes=c("F32","F33")

#http://www.icd9data.com/2015/Volume1/240-279/249-259/default.htm
#https://www.icd10data.com/ICD10CM/Codes/E00-E89/E08-E13
diabetes_icd_9_codes=c("250")
diabetes_icd_10_codes=c("E08","E09","E10","E11","E13")

census_and_state_codes=read.csv("./results_mar/census_and_state_codes.csv")
census_and_state_codes[1:9,"state_code"] = paste("0",census_and_state_codes[1:9,"state_code"],sep="")

race_codes=data.frame(race_code = seq(0,6),
                      race = c("Unknown", "White", "Black", "Other", "Asian", "Hispanic", "North American Native")
                      )

sex_codes=data.frame(sex_code = seq(0,2),
                    sex = c("Unknown","Male","Female"))
                      
                      
                      

# Comparisons and analyses

## Comparisons between patients treated by integrated vs independent docotrs

I will compare the patients who were treated by the integrated vs non-integrated physicians, cardiologists and interventionists during the study period. Year 2013 did not divide cardiologists and inteventionists. So, We will not include this year in our comparisons of cardiologists and inteventionists.



## Reading the patient and physician integration results

In [3]:
yearly_calculations_stable_angina =
read_fst("results_apr/yearly_calculations_stable_angina_with_integration.fst"
         ,as.data.table = T) 
yearly_calculations_unstable_angina =
read_fst("results_apr/yearly_calculations_unstable_angina_with_integration.fst"
         ,as.data.table = T)
#physician_integration_stats = 
#read_fst("results_feb/physician_integration_stats.fst"
#        ,as.data.table = T)
#melted_physician_integration_stats=
#read_fst("results_feb/melted_physician_integration_stats.fst"
#        ,as.data.table = T)

In [4]:
tail(yearly_calculations_stable_angina,5)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,year_first_diagnosed,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLE

## Adding sex, race, and state names from codes

In [5]:
add_personal_details=function(data){
    
  require(tidyverse)
  require(dtplyr)
  require(lubridate)
  
  result=data %>%
  left_join(.,census_and_state_codes[,-1],by=c("state_code_at_diagnosis"="state_code"))%>%
  left_join(.,race_codes,by=c("race_code_at_diagnosis"="race_code"))%>%
  left_join(.,sex_codes,by=c("sex_code_at_diagnosis"="sex_code"))%>%
  mutate(
    age_group=case_when(
      age_at_diagnosis<75 & age_at_diagnosis>64 ~ "65-74",
      age_at_diagnosis>74 & age_at_diagnosis<85 ~ "75-84",
      age_at_diagnosis>84 ~ "85+"
    )
  )%>%
  as.data.table()
  
  return(result)
}

In [6]:
yearly_calculations_stable_angina=add_personal_details(yearly_calculations_stable_angina)
yearly_calculations_unstable_angina=add_personal_details(yearly_calculations_unstable_angina)

Loading required package: lubridate


Attaching package: ‘lubridate’


The following object is masked from ‘package:reshape’:

    stamp


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [7]:
data_for_modelling_filter=function(data){
  library(tidyverse)
  library(dtplyr)
  
  data%>%
  filter(state_code_at_diagnosis %!in% non_us_state_codes &
         age_at_diagnosis>=65&
         ((died_in_one_year_after_diagnosis==T & HMO_INDICATOR_sum>0) |
          (died_in_one_year_after_diagnosis==F & HMO_INDICATOR_sum==12))&
         ((died_in_one_year_after_diagnosis==T & ENTITLEMENT_BUY_IN_IND_sum>0) |
          (died_in_one_year_after_diagnosis==F & ENTITLEMENT_BUY_IN_IND_sum==12))&
         year_first_diagnosed!=2013&
         !is.na(most_common_cardiologist_PRF_PHYSN_NPI)
        )%>%
  as.data.table()
}

In [8]:
yearly_calculations_stable_angina=data_for_modelling_filter(yearly_calculations_stable_angina)
yearly_calculations_unstable_angina=data_for_modelling_filter(yearly_calculations_unstable_angina)

In [10]:
colnames(yearly_calculations_stable_angina)

[1] "DESY_SORT_KEY"                                                                   "first_diagnosis"                                                                 "tot_allowed_carrier"                                                            
  [4] "office_visit_count"                                                              "office_visit_cost_carrier"                                                       "cardiology_visit_count"                                                         
  [7] "distinct_clinicians"                                                             "distinct_cardiologists"                                                          "distinct_primary_care_physicians"                                               
 [10] "distinct_surgeons"                                                               "distinct_other_specialties"                                                      "catheterization_count"                                                          
 [13] "catheterization_cost_carrier"                                                    "ecg_count"                                                                       "ecg_cost_carrier"                                                               
 [16] "cardiac_ct_count"                                                                "cardiac_ct_cost_carrier"                                                         "cardiac_mri_count"                                                              
 [19] "cardiac_mri_cost_carrier"                                                        "stress_test_count"                                                               "stress_test_cost_carrier"                                                       
 [22] "echocardiography_count"                                                          "echocardiography_cost_carrier"                                                   "angioplasty_count"                                                              
 [25] "angioplasty_cost_carrier"                                                        "CABG_count"                                                                      "CABG_cost_carrier"                                                              
 [28] "stable_angina"                                                                   "unstable_angina"                                                                 "MI"                                                                             
 [31] "cardiac_arrest"                                                                  "stroke"                                                                          "hypertension"                                                                   
 [34] "copd"                                                                            "obesity"                                                                         "depression"                                                                     
 [37] "diabetes"                                                                        "icd_9_pure"                                                                      "icd_10_pure"                                                                    
 [40] "year_first_diagnosed"                                                            "tot_allowed_carrier_cardiology_related"                                          "office_visit_count_cardiology_related"                                          
 [43] "office_visit_cost_carrier_cardiology_related"                                    "cardiology_visit_count_cardiology_related"                                       "distinct_clinicians_cardiology_related"                                         
 [46] "distinct_cardiologists_cardiology_related"                                       "distinct_primary_care_physicians_cardiology_related"                             "distinct_surgeons_cardiology_related"              

In [11]:
length(unique(yearly_calculations_stable_angina$DESY_SORT_KEY))

[1] 15316

In [12]:
length(unique(yearly_calculations_unstable_angina$DESY_SORT_KEY))

[1] 9691

In [13]:
model_for_vi_prediction=lm(cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS>0.5~primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS>0.5,
   data = yearly_calculations_stable_angina)

In [14]:
summary(model_for_vi_prediction)


Call:
lm(formula = cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS > 
    0.5 ~ primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS > 
    0.5, data = yearly_calculations_stable_angina)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.5768 -0.1859 -0.1859 -0.1859  0.8141 

Coefficients:
                                                                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)                                                           0.185911   0.004066   45.72   <2e-16 ***
primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS > 0.5TRUE 0.390918   0.014493   26.97   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3983 on 10414 degrees of freedom
  (4900 observations deleted due to missingness)
Multiple R-squared:  0.0653,	Adjusted R-squared:  0.06521 
F-statistic: 727.5 on 1 and 10414 DF,  p-value: < 2.2e-16


In [135]:
library(jtools)
library(knitr)
library(kableExtra)

In [192]:
naive_model=lm(catheterization_count>0~
               (cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS>0.5)+
               sex+
               race+
               age_at_diagnosis+
               census_region+
               hypertension+
               copd+
               obesity+
               depression+
               diabetes+
               wscore,
               data = yearly_calculations_stable_angina
              )



In [193]:
summ(naive_model)

MODEL INFO:
Observations: 12549 (2767 missing obs. deleted)
Dependent Variable: catheterization_count > 0
Type: OLS linear regression 

MODEL FIT:
F(18,12530) = 34.18, p = 0.00
R² = 0.05
Adj. R² = 0.05 

Standard errors: OLS
----------------------------------------------------------------------------
                                                                 Est.   S.E.
------------------------------------------------------------- ------- ------
(Intercept)                                                      1.04   0.06
cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS        0.05   0.01
> 0.5TRUE                                                                   
sexMale                                                          0.06   0.01
raceBlack                                                        0.05   0.03
raceHispanic                                                     0.05   0.04
raceNorth American Native                                        0.15   0.07
raceO

## IV model

In [149]:
tail(yearly_calculations_stable_angina,20)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,year_first_diagnosed,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLE

In [162]:
sum(yearly_calculations_stable_angina[,primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS]<0.5,na.rm=T)
sum(yearly_calculations_unstable_angina[,primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS]<0.5,na.rm=T)


[1] 9591

[1] 6067

In [202]:
iv_model=lm((cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS>=0.5)~
               primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_half+
               sex+
               race+
               age_at_diagnosis+
               census_region+
               hypertension+
               copd+
               obesity+
               depression+
               diabetes+
               wscore,
               data = yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.5,]
              )

In [203]:
summ(iv_model)

MODEL INFO:
Observations: 6768 (2823 missing obs. deleted)
Dependent Variable: (cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS >= 0.5)
Type: OLS linear regression 

MODEL FIT:
F(18,6749) = 16.72, p = 0.00
R² = 0.04
Adj. R² = 0.04 

Standard errors: OLS
-------------------------------------------------------------------------------------
                                                                                 Est.
----------------------------------------------------------------------------- -------
(Intercept)                                                                      0.25
primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_halfTRUE          0.23
sexMale                                                                         -0.00
raceBlack                                                                        0.06
raceHispanic                                                                    -0.01
raceNorth American Native                       

In [182]:
yearly_calculations_stable_angina[,primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_half
                                  :=(primary_care_physician_2013_avg_in_year_of_diagnosis_non_exclusive_HCPCS>=0.5)
                                 ]

In [204]:
predictions_iv=predict(iv_model,
        newdata = yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.5,
                                                    .(primary_care_physician_2013_avg_in_year_of_diagnosis_more_than_half,
                                                    sex,
                                                    race,
                                                    age_at_diagnosis,
                                                    census_region,
                                                    hypertension,
                                                    copd,
                                                    obesity,
                                                    depression,
                                                    diabetes,
                                                    wscore)])

In [205]:
head(predictions_iv)

1         2         3         4         5         6 
0.1669053 0.2104425 0.1848067 0.2079860 0.1733503        NA

In [206]:
yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.5,predictions_iv:=predictions_iv]

In [207]:
iv_model_stage_2=lm(catheterization_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.5]
                   )

In [208]:
  summ(iv_model_stage_2, robust = "HC1")

MODEL INFO:
Observations: 6771 (2820 missing obs. deleted)
Dependent Variable: catheterization_count > 0
Type: OLS linear regression 

MODEL FIT:
F(18,6752) = 19.82, p = 0.00
R² = 0.05
Adj. R² = 0.05 

Standard errors: Robust, type = HC1
--------------------------------------------------------------
                                   Est.   S.E.   t val.      p
------------------------------- ------- ------ -------- ------
(Intercept)                        1.14   0.08    14.65   0.00
predictions_iv                     0.23   0.10     2.35   0.02
sexMale                            0.05   0.01     4.30   0.00
raceBlack                          0.03   0.04     0.67   0.50
raceHispanic                       0.10   0.06     1.65   0.10
raceNorth American Native          0.10   0.11     0.89   0.37
raceOther                          0.01   0.05     0.16   0.87
raceUnknown                        0.03   0.08     0.42   0.67
raceWhite                          0.09   0.03     2.67   0.01
age_at

In [218]:
iv_model_stage_2=lm(catheterization_count>0~
                    predictions_iv+
                    sex+
                    race+
                    age_at_diagnosis+
                    census_region+
                    hypertension+
                    copd+
                    obesity+
                    depression+
                    diabetes+
                    wscore,
                    data = yearly_calculations_stable_angina[primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS<0.5]
                   )

In [219]:
  summ(iv_model_stage_2, robust = "HC1")

MODEL INFO:
Observations: 6771 (2820 missing obs. deleted)
Dependent Variable: catheterization_count > 0
Type: OLS linear regression 

MODEL FIT:
F(18,6752) = 19.82, p = 0.00
R² = 0.05
Adj. R² = 0.05 

Standard errors: Robust, type = HC1
--------------------------------------------------------------
                                   Est.   S.E.   t val.      p
------------------------------- ------- ------ -------- ------
(Intercept)                        1.14   0.08    14.65   0.00
predictions_iv                     0.23   0.10     2.35   0.02
sexMale                            0.05   0.01     4.30   0.00
raceBlack                          0.03   0.04     0.67   0.50
raceHispanic                       0.10   0.06     1.65   0.10
raceNorth American Native          0.10   0.11     0.89   0.37
raceOther                          0.01   0.05     0.16   0.87
raceUnknown                        0.03   0.08     0.42   0.67
raceWhite                          0.09   0.03     2.67   0.01
age_at

In [113]:
model_for_vi_prediction=lm(cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS>0.5~primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS>0.5,
   data = yearly_calculations_stable_angina)



In [114]:
summary(model_for_vi_prediction)


Call:
lm(formula = cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS > 
    0.5 ~ primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS > 
    0.5, data = yearly_calculations_stable_angina)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.5528 -0.1796 -0.1796 -0.1796  0.8204 

Coefficients:
                                                                      Estimate
(Intercept)                                                            0.17957
primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS > 0.5TRUE  0.37321
                                                                      Std. Error
(Intercept)                                                              0.00313
primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS > 0.5TRUE    0.01098
                                                                      t value
(Intercept)                                                             57.38
primary_care_physician_2013_avg_in_2013_non_exclus

In [106]:
data.frame(summary_naive_model[,c("coefficients",])

,naive_model.coefficients
,<dbl>
(Intercept),0.4620868369
cardiologist_avg_in_year_of_diagnosis_non_exclusive_HCPCS > 0.5TRUE,0.0057857648
sexMale,-0.0082758015
raceBlack,-0.0079291916
raceHispanic,-0.0535267498
raceNorth American Native,-0.0166116322
raceOther,0.0068311557
raceUnknown,-0.0137014046
raceWhite,-0.0602084491


In [126]:
data_for_analysis_stable_angina=
yearly_calculations_stable_angina%>%
filter(state_code_at_diagnosis %!in% non_us_state_codes &
          HMO_INDICATOR_sum == 12 &
          ((died_in_one_year_after_diagnosis==T & ENTITLEMENT_BUY_IN_IND_sum>0) |
           (died_in_one_year_after_diagnosis==F & ENTITLEMENT_BUY_IN_IND_sum==12))&
       !is.na(primary_care_physician_2013_avg_in_2013_non_exclusive_HCPCS))%>%
   as.data.table()


In [127]:
nrow(data_for_analysis_stable_angina)

[1] 19852

In [121]:
nrow(yearly_calculations_stable_angina)

[1] 38426